In [7]:
import os
import numpy as np
import gin
import torch
from torch.nn.functional import interpolate
from PIL import Image
import sys, os
sys.path.insert(0, os.path.abspath('../../'))

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from network import network
from image_retrieval import rank_images
from network import network
from pose_prediction import predictor
from datasets import base_dataset

In [10]:
from pose_prediction.sparse_to_dense_featuremetric_predictor import SparseToDenseFeatureMetricPnP
from datasets import dataload_helpers as data_helpers

In [11]:
DATA_PATH = "/home/aunagar/Personal/Others/learning/feature detection/robotcar/"

# put triangulation file in the same folder as robotcar data
triangulation_file = DATA_PATH + 'robotcar_triangulation.npz'
nvm_filepath = DATA_PATH + '3D-models/all-merged/all.nvm'
image_root = DATA_PATH + 'images/'

In [12]:
ref_image = [image_root + 'overcast-reference/rear/1417176458999821.jpg']
query_image = [image_root + 'overcast-reference/rear/1417176511810439.jpg']

filenames = ref_image + query_image

In [13]:
filename_to_pose = data_helpers.from_nvm(nvm_filepath, filenames)
filename_to_local_reconstruction = data_helpers.load_triangulation_data(triangulation_file, filenames)
filename_to_intrinsics = data_helpers.load_intrinsics(filenames)

Found  overcast-reference/rear/1417176458999821.jpg
Found  overcast-reference/rear/1417176511810439.jpg


In [14]:
##### You can ignore this part #####
@gin.configurable
def get_dataset_loader(dataset_loader_cls):
    return dataset_loader_cls

@gin.configurable
def get_pose_predictor(pose_predictor_cls: predictor.PosePredictor,
                       dataset: base_dataset.BaseDataset,
                       network: network.ImageRetrievalModel,
                       ranks: np.ndarray,
                       log_images: bool):
    return pose_predictor_cls(dataset=dataset,
                              network=network,
                              ranks=ranks,
                              log_images=log_images)

In [15]:
# Load gin config based on dataset name
gin.parse_config_file(
    'configs/runs/run_{}_on_{}.gin'.format('sparse_to_dense', 'robotcar'))

In [16]:
net = network.ImageRetrievalModel(device = "cpu")

In [17]:
s2dPnP = SparseToDenseFeatureMetricPnP(filename_to_pose, filename_to_intrinsics, filename_to_local_reconstruction, net)

In [18]:
solved_case, query_hypercolumn, reference_hypercolumn = s2dPnP.run(query_image[0], ref_image[0])

>> Generating pose predictions using sparse-to-dense matching...
Hypercolumn size is 2048
Hypercolumn resolution is 128 x 128
Hypercolumn size is 2048
Hypercolumn resolution is 128 x 128


In [19]:
from BA.featureBA.src.model_philipp import sparse3DBA
from BA.featureBA.src.utils import sobel_filter

In [20]:
pts3D = torch.from_numpy(solved_case.points_3d.reshape(-1,3))
ref2d = torch.flip(torch.from_numpy((1/8*solved_case.reference_inliers).astype(int)),(1,))
feature_ref = torch.cat([reference_hypercolumn.squeeze(0)[:, i, j].unsqueeze(0) for i, j in zip(ref2d[:,0],
                        ref2d[:,1])]).type(torch.DoubleTensor)
feature_map_query = query_hypercolumn.squeeze(0).type(torch.DoubleTensor)

In [15]:
T_init = solved_case.m
R_init, t_init = torch.from_numpy(T_init[:3, :3]), torch.from_numpy(T_init[:3,3])
feature_grad_x, feature_grad_y = sobel_filter(feature_map_query)
K = torch.from_numpy(filename_to_intrinsics[ref_image[0]][0]).type(torch.DoubleTensor)

In [20]:
feature_map_query.shape

torch.Size([2048, 128, 128])

In [18]:
model = sparse3DBA(n_iters = 50, lambda_ = 0, verbose=True)
R, t = model(pts3D, feature_ref, feature_map_query, feature_grad_x, feature_grad_y, K, R_init, t_init)

printing
Iter  0 0.667959813353738
dr is :  tensor([-0.0026,  0.0003,  0.0010], dtype=torch.float64)
dt is :  tensor([ 0.0068, -0.0155, -0.0144], dtype=torch.float64)
new cost is  0.6668234929510498
printing
Iter  1 0.6668234929510498
dr is :  tensor([-2.7503e-04,  1.4125e-05,  8.2066e-05], dtype=torch.float64)
dt is :  tensor([ 0.0009, -0.0016, -0.0016], dtype=torch.float64)
new cost is  0.6668199179366404
printing
Iter  2 0.6668199179366404
dr is :  tensor([-2.7922e-04,  1.7194e-05,  8.2416e-05], dtype=torch.float64)
dt is :  tensor([ 0.0009, -0.0017, -0.0016], dtype=torch.float64)
new cost is  0.6668797091069667
cost increased, continue with next iteration
printing
Iter  3 0.6668199179366404
dr is :  tensor([-2.7921e-05,  1.7195e-06,  8.2414e-06], dtype=torch.float64)
dt is :  tensor([ 8.6187e-05, -1.6707e-04, -1.5829e-04], dtype=torch.float64)
new cost is  0.6667130809724274
printing
Iter  4 0.6667130809724274
dr is :  tensor([-2.8402e-04,  1.4585e-05,  8.0733e-05], dtype=torch.flo

dr is :  tensor([-7.7367e-09,  7.8753e-09,  8.1879e-09], dtype=torch.float64)
dt is :  tensor([ 1.0752e-07, -2.9643e-10, -1.3356e-08], dtype=torch.float64)
new cost is  0.6667130809724274
printing
Iter  34 0.6667130809724274
dr is :  tensor([-7.6771e-07,  7.7285e-07,  8.0585e-07], dtype=torch.float64)
dt is :  tensor([ 1.0578e-05, -6.9694e-08, -1.4330e-06], dtype=torch.float64)
new cost is  0.6668862135573637
cost increased, continue with next iteration
printing
Iter  35 0.6667130809724274
dr is :  tensor([-7.7367e-09,  7.8753e-09,  8.1879e-09], dtype=torch.float64)
dt is :  tensor([ 1.0752e-07, -2.9643e-10, -1.3356e-08], dtype=torch.float64)
new cost is  0.6667130809724274
printing
Iter  36 0.6667130809724274
dr is :  tensor([-7.6771e-07,  7.7285e-07,  8.0585e-07], dtype=torch.float64)
dt is :  tensor([ 1.0578e-05, -6.9694e-08, -1.4330e-06], dtype=torch.float64)
new cost is  0.6668862135573637
cost increased, continue with next iteration
printing
Iter  37 0.6667130809724274
dr is :  t

In [51]:
p3d = torch.mm(R_init, pts3D.T).T + t_init
p2d = torch.mm(K, p3d.T).T

In [52]:
p2d = p2d/p2d[:,2,None]

In [53]:
ref2d

tensor([[ 32,  44],
        [ 47,  54],
        [ 26,  51],
        [ 42,  51],
        [ 41,  44],
        [ 61,  48],
        [ 70,  52],
        [ 61,  60],
        [ 23,  56],
        [ 44,  52],
        [ 24,  39],
        [ 50,  48],
        [ 75,  59],
        [  5,  32],
        [ 65,  61],
        [ 87,  63],
        [ 41,  43],
        [ 87,  66],
        [ 19,  30],
        [ 18,  58],
        [ 29,  42],
        [ 44,  59],
        [ 54,  47],
        [ 17,  50],
        [ 60,  44],
        [ 69,  62],
        [ 78,  62],
        [ 53,  33],
        [ 82,  32],
        [ 41,  58],
        [  2,  55],
        [ 35,  46],
        [ 89,  63],
        [ 71,  49],
        [ 10,  58],
        [114,  27],
        [ 36,  43],
        [  5,  62],
        [ 71,  53],
        [ 66,  48],
        [ 25,  53],
        [ 80,  31],
        [ 17,  39],
        [ 47,  47],
        [  8,  41],
        [ 72,  29],
        [ 57,  60],
        [ 23,  59],
        [ 62,  53],
        [ 48,  47],


In [22]:
filename_to_pose

{'overcast-reference/rear/1417176458999821.jpg': [[0.46762190460884,
   -0.38178292214377,
   0.50118889778761,
   0.61998487355402],
  array([[-0.27114311, -0.96252774, -0.00466546, -0.10964306],
         [ 0.19714629, -0.06027889,  0.97851919,  1.02252028],
         [-0.94213309,  0.26439896,  0.20610298, -0.48196902],
         [ 0.        ,  0.        ,  0.        ,  1.        ]])],
 'overcast-reference/rear/1417176511810439.jpg': [[0.40197008393349,
   -0.32121406706814,
   0.54692211416876,
   0.66039213783611],
  array([[-0.47048315, -0.88227392,  0.01543817,  0.01947696],
         [ 0.17955761, -0.07859251,  0.98060302,  0.55960274],
         [-0.86394715,  0.46412924,  0.19539545,  1.97999074],
         [ 0.        ,  0.        ,  0.        ,  1.        ]])]}

In [119]:
solved_case.reference_inliers

array([[257., 359.],
       [377., 432.],
       [212., 408.],
       [343., 412.],
       [330., 356.],
       [495., 385.],
       [560., 418.],
       [495., 481.],
       [186., 451.],
       [354., 420.],
       [192., 317.],
       [400., 384.],
       [606., 476.],
       [ 47., 259.],
       [521., 491.],
       [697., 504.],
       [332., 344.],
       [700., 534.],
       [156., 243.],
       [145., 469.],
       [239., 342.],
       [357., 477.],
       [434., 376.],
       [139., 401.],
       [485., 352.],
       [557., 498.],
       [628., 502.],
       [427., 264.],
       [660., 257.],
       [331., 471.],
       [ 19., 444.],
       [287., 371.],
       [719., 508.],
       [568., 394.],
       [ 81., 464.],
       [919., 223.],
       [288., 351.],
       [ 41., 498.],
       [573., 429.],
       [529., 389.],
       [204., 426.],
       [646., 255.],
       [141., 316.],
       [377., 377.],
       [ 64., 331.],
       [579., 237.],
       [461., 483.],
       [186.,